# Command and Control Application Suite E2E Demonstration

© Crown-owned copyright 2024, Defence Science and Technology Laboratory UK

This notebook demonstrates the current implementation of the command and control (C2) server and beacon applications in primAITE.

In [ ]:
# Imports
import yaml
from primaite.config.load import data_manipulation_config_path
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.system.applications.red_applications.c2.c2_beacon import C2Beacon
from primaite.simulator.system.applications.red_applications.c2.c2_server import C2Server
from primaite.simulator.system.applications.red_applications.c2.abstract_c2 import C2Command
from primaite.simulator.system.applications.red_applications.ransomware_script import RansomwareScript
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server

## **Notebook Setup** | **Network Configuration:**

This notebook uses the same network setup as UC2. Please refer to the main [UC2-E2E-Demo notebook for further reference](./Data-Manipulation-E2E-Demonstration.ipynb).

However, this notebook replaces the red agent used in UC2 with a custom proxy red agent built for this notebook.

In [ ]:
custom_c2_agent = """
  - ref: CustomC2Agent
    team: RED
    type: ProxyAgent
    observation_space: null
    action_space:
      action_list:
        - type: DONOTHING
        - type: NODE_APPLICATION_INSTALL
        - type: NODE_APPLICATION_EXECUTE
        - type: CONFIGURE_C2_BEACON
        - type: C2_SERVER_RANSOMWARE_LAUNCH
        - type: C2_SERVER_RANSOMWARE_CONFIGURE
        - type: C2_SERVER_TERMINAL_COMMAND
        - type: C2_SERVER_DATA_EXFILTRATE
      options:
        nodes:
          - node_name: web_server
            applications: 
              - application_name: C2Beacon
          - node_name: client_1
            applications: 
              - application_name: C2Server
        max_folders_per_node: 1
        max_files_per_folder: 1
        max_services_per_node: 2
        max_nics_per_node: 8
        max_acl_rules: 10
        ip_list:
          - 192.168.1.21
          - 192.168.1.14
        wildcard_list:
          - 0.0.0.1
      action_map:
        0:
          action: DONOTHING
          options: {}
        1:
          action: NODE_APPLICATION_INSTALL
          options:
            node_id: 0
            application_name: C2Beacon
        2:
          action: CONFIGURE_C2_BEACON
          options:
            node_id: 0
            config:
              c2_server_ip_address: 192.168.10.21
              keep_alive_frequency:
              masquerade_protocol:
              masquerade_port:
        3:
          action: NODE_APPLICATION_EXECUTE
          options:
            node_id: 0
            application_id: 0  
        4:
          action: C2_SERVER_TERMINAL_COMMAND
          options:
            node_id: 1
            ip_address:
            account:
              username: admin
              password: admin
            commands:
              - 
                - software_manager
                - application
                - install
                - RansomwareScript
        5:
          action: C2_SERVER_RANSOMWARE_CONFIGURE
          options:
            node_id: 1
            config:
              server_ip_address: 192.168.1.14
              payload: ENCRYPT
        6:
          action: C2_SERVER_DATA_EXFILTRATE
          options:
            node_id: 1
            target_file_name: "database.db"
            target_folder_name: "database"
            exfiltration_folder_name: "spoils"
            target_ip_address: 192.168.1.14
            account:
              username: admin
              password: admin 

        7:
          action: C2_SERVER_RANSOMWARE_LAUNCH
          options:
            node_id: 1
        8:
          action: CONFIGURE_C2_BEACON
          options:
            node_id: 0
            config:
              c2_server_ip_address: 192.168.10.21
              keep_alive_frequency: 10
              masquerade_protocol: TCP
              masquerade_port: DNS
        9:
          action: CONFIGURE_C2_BEACON
          options:
            node_id: 0
            config:
              c2_server_ip_address: 192.168.10.22
              keep_alive_frequency:
              masquerade_protocol:
              masquerade_port:

    reward_function:
      reward_components:
        - type: DUMMY
"""
c2_agent_yaml = yaml.safe_load(custom_c2_agent)

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = c2_agent_yaml
    

env = PrimaiteGymEnv(env_config=cfg)

## **Notebook Setup** | Network Prerequisites

Before the Red Agent is able to perform any C2 specific actions, the C2 Server needs to be installed and run.
This is because in higher fidelity environments (and the real-world) a C2 server would not be accessible by a private network blue agent and the C2 Server would already be in place before the an adversary (Red Agent) starts.

The cells below install and run the C2 Server on client_1 directly via the simulation API.

In [ ]:
client_1: Computer = env.game.simulation.network.get_node_by_hostname("client_1")
client_1.software_manager.install(C2Server)
c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server.run()
client_1.software_manager.show()

## **Command and Control** | C2 Beacon Actions

Before a C2 Server can accept any commands it must first establish connection with a C2 Beacon.

A red agent is able to install, configure and establish a C2 beacon at any point in an episode. The code cells below demonstrate the actions and option parameters that are needed to perform this.

### **Command and Control** | C2 Beacon Actions | NODE_APPLICATION_INSTALL

The custom proxy red agent defined at the start of this notebook has been configured to install the C2 Beacon as action ``1`` in it's action map. 

The below yaml snippet shows all the relevant agent options for this action:

```yaml
    action_space:
      action_list:
      ...
        - type: NODE_APPLICATION_INSTALL
      ...
      options:
        nodes: # Node List
          - node_name: web_server
            applications: 
              - application_name: C2Beacon
        ...
    ...
      action_map:
        1:
          action: NODE_APPLICATION_INSTALL 
          options:
            node_id: 0 # Index 0 at the node list.
            application_name: C2Beacon
```

In [ ]:
env.step(1)
web_server: Computer = env.game.simulation.network.get_node_by_hostname("web_server")
web_server.software_manager.show()

### **Command and Control** | C2 Beacon Actions | CONFIGURE_C2_BEACON 

The custom proxy red agent defined at the start of this notebook can configure the C2 Beacon via action ``2`` in it's action map. 

The yaml snippet below shows all the relevant agent options for this action:

```yaml
    action_space:
      action_list:
      ...
        - type: CONFIGURE_C2_BEACON
      ...
      options:
        nodes: # Node List
          - node_name: web_server
        ...
    ...
      action_map:
        ...
        2:
          action: CONFIGURE_C2_BEACON
          options:
            node_id: 0 # Node Index
            config: # Further information about these config options can be found at the bottom of this notebook.
              c2_server_ip_address: 192.168.10.21
              keep_alive_frequency:
              masquerade_protocol:
              masquerade_port:
```

In [ ]:
env.step(2)
c2_beacon: C2Beacon = web_server.software_manager.software["C2Beacon"]
web_server.software_manager.show()
c2_beacon.show()

### **Command and Control** | C2 Beacon Actions | NODE_APPLICATION_EXECUTE

The final action is ``NODE_APPLICATION_EXECUTE`` which is used to establish a connection for the C2 application. This action can be called by the Red Agent via action ``3`` in it's action map. 

The yaml snippet below shows all the relevant agent options for this action:

```yaml
    action_space:
      action_list:
      ...
        - type: NODE_APPLICATION_EXECUTE
      ...
      options:
        nodes: # Node List
          - node_name: web_server
              applications: 
              - application_name: C2Beacon
        ...
    ...
      action_map:
        ...
        3:
          action: NODE_APPLICATION_EXECUTE
          options:
            node_id: 0
            application_id: 0
```

In [ ]:
env.step(3) 

In [ ]:
c2_beacon.show()
c2_server.show()

## **Command and Control** | C2 Server Actions

Once the C2 suite has been successfully established, the C2 Server based actions become available to the Red Agent. 


This next section will demonstrate the different actions that become available to a red agent after establishing a C2 connection:

### **Command and Control** | C2 Server Actions | C2_SERVER_TERMINAL_COMMAND

The C2 Server's terminal action: ``C2_SERVER_TERMINAL_COMMAND`` is indexed at  ``4`` in it's action map. 

This action leverages the terminal service that is installed by default on all nodes to grant red agents a lot more configurability. If you're unfamiliar with terminals then it's recommended that you refer to the ``Terminal Processing`` notebook.

It's worth noting that an additional benefit a red agent has when using the terminal service via the C2 Server is that you can execute multiple commands in one action. 

In this notebook, the ``C2_SERVER_TERMINAL_COMMAND`` is used to install a RansomwareScript application on the ``web_server`` node.

The yaml snippet below shows all the relevant agent options for this action:

``` yaml
    action_space:
      action_list:
      ...
      - type: C2_SERVER_TERMINAL_COMMAND
      ...
      options:
        nodes: # Node List
        ...
        - node_name: client_1
            applications: 
              - application_name: C2Server
        ...
        action_map:
           4:
            action: C2_SERVER_TERMINAL_COMMAND
            options:
              node_id: 1
              ip_address:
              account:
              username: admin
              password: admin
              commands:
              - 
                  - software_manager
                  - application
                  - install
                  - RansomwareScript
```

In [ ]:
env.step(4)

In [ ]:
client_1.software_manager.show()

### **Command and Control** | C2 Server Actions | C2_SERVER_RANSOMWARE_CONFIGURE

Another action the C2 Server grants is the ability for a Red Agent to configure the RansomwareScript via the C2 Server rather than the note directly.

This action is indexed as action ``5``.

The yaml snippet below shows all the relevant agent options for this action:

``` yaml
    action_space:
      action_list:
      ...
      - type: C2_SERVER_RANSOMWARE_CONFIGURE
      ...
      options:
        nodes: # Node List
        ...
        - node_name: client_1
            applications: 
              - application_name: C2Server
        ...
        action_map:
           5:
            action: C2_SERVER_RANSOMWARE_CONFIG
            options:
              node_id: 1
              config:
                server_ip_address: 192.168.1.14
                payload: ENCRYPT
```


In [ ]:
env.step(5)

In [ ]:
ransomware_script: RansomwareScript = web_server.software_manager.software["RansomwareScript"]
web_server.software_manager.show()
ransomware_script.show()

### **Command and Control** | C2 Server Actions | C2_SERVER_DATA_EXFILTRATE

The second to last action available is the ``C2_SERVER_DATA_EXFILTRATE`` which is indexed as action ``6`` in the action map.

This action can be used to exfiltrate a target file on a remote node to the C2 Beacon and the C2 Server's host file system via the ``FTP`` services.

The below yaml snippet shows all the relevant agent options for this action:

``` yaml
    action_space:
      action_list:
      ...
      - type: C2_SERVER_DATA_EXFILTRATE
      ...
      options:
        nodes: # Node List
        ...
        - node_name: client_1
            applications: 
              - application_name: C2Server
        ...
        action_map:
           6:
            action: C2_SERVER_DATA_EXFILTRATE
            options:
              node_id: 1
              target_file_name: "database.db"
              target_folder_name: "database"
              exfiltration_folder_name: "spoils"
              target_ip_address: "192.168.1.14"
              account:
                username: "admin",
                password: "admin"

```

In [ ]:
env.step(6)

In [ ]:
client_1: Computer = env.game.simulation.network.get_node_by_hostname("client_1")
client_1.software_manager.file_system.show(full=True)

In [ ]:
web_server: Computer = env.game.simulation.network.get_node_by_hostname("web_server")
web_server.software_manager.file_system.show(full=True)

### **Command and Control** | C2 Server Actions | C2_SERVER_RANSOMWARE_LAUNCH

Finally, the last available action is for the C2_SERVER_RANSOMWARE_LAUNCH to start the ransomware script installed on the same node as the C2 beacon.

This action is indexed as action ``7``.

"The yaml snippet below shows all the relevant agent options for this action:

``` yaml
    action_space:
      action_list:
      ...
      - type: C2_SERVER_RANSOMWARE_LAUNCH
      ...
      options:
        nodes: # Node List
        ...
        - node_name: client_1
            applications: 
              - application_name: C2Server
        ...
        action_map:
           7:
            action: C2_SERVER_RANSOMWARE_LAUNCH
            options:
              node_id: 1
```


In [ ]:
env.step(7)

In [ ]:
database_server: Server = env.game.simulation.network.get_node_by_hostname("database_server")
database_server.software_manager.file_system.show(full=True)

## **Command and Control** | Blue Agent Relevance

The next section of the notebook will demonstrate the impact the command and control suite has on the Blue Agent's observation space as well as some potential actions that can be used to prevent the attack from being successful.

The code cell below recreates the UC2 network and swaps out the previous custom red agent with a custom blue agent. 

In [ ]:
custom_blue_agent_yaml = """  
  - ref: defender
    team: BLUE
    type: ProxyAgent

    observation_space:
      type: CUSTOM
      options:
        components:
          - type: NODES
            label: NODES
            options:
              hosts:
                - hostname: web_server
                  applications:
                    - application_name: C2Beacon
                    - application_name: RansomwareScript
                  folders:
                    - folder_name: exfiltration_folder
                      files:
                      - file_name: database.db
                - hostname: database_server
                  folders:
                    - folder_name: exfiltration_folder
                      files:
                      - file_name: database.db
                - hostname: client_1
                - hostname: client_2
              num_services: 0
              num_applications: 2
              num_folders: 1
              num_files: 1
              num_nics: 1
              include_num_access: true
              include_nmne: false
              monitored_traffic:
                icmp:
                    - NONE
                tcp:
                    - HTTP
                    - DNS
                    - FTP
              routers:
                - hostname: router_1
              num_ports: 3
              ip_list:
                - 192.168.1.10
                - 192.168.1.12
                - 192.168.1.14
                - 192.168.1.16
                - 192.168.1.110
                - 192.168.10.21
                - 192.168.10.22
                - 192.168.10.110
              wildcard_list:
                - 0.0.0.1
              port_list:
                - 80
                - 53
                - 21
              protocol_list:
                - ICMP
                - TCP
                - UDP
              num_rules: 10

          - type: LINKS
            label: LINKS
            options:
              link_references:
                - router_1:eth-1<->switch_1:eth-8
                - router_1:eth-2<->switch_2:eth-8
                - switch_1:eth-1<->web_server:eth-1
                - switch_1:eth-2<->web_server:eth-1
                - switch_1:eth-3<->database_server:eth-1
                - switch_1:eth-4<->backup_server:eth-1
                - switch_1:eth-7<->security_suite:eth-1
                - switch_2:eth-1<->client_1:eth-1
                - switch_2:eth-2<->client_2:eth-1
                - switch_2:eth-7<->security_suite:eth-2
          - type: "NONE"
            label: ICS
            options: {}
    
    action_space:
      action_list:
        - type: NODE_APPLICATION_REMOVE
        - type: NODE_SHUTDOWN
        - type: ROUTER_ACL_ADDRULE
        - type: DONOTHING
      action_map:
          0:
            action: DONOTHING
            options: {}
          1:
            action: NODE_APPLICATION_REMOVE
            options:
              node_id: 0
              application_name: C2Beacon
          2:
            action: NODE_SHUTDOWN
            options:
              node_id: 0
          3:
            action: ROUTER_ACL_ADDRULE
            options:
              target_router: router_1
              position: 1
              permission: 2
              source_ip_id: 2
              dest_ip_id: 3
              source_port_id: 2
              dest_port_id: 2
              protocol_id: 1
              source_wildcard_id: 0
              dest_wildcard_id: 0  


      options:
        nodes:
        - node_name: web_server
          applications:
          - application_name: C2Beacon

        - node_name: database_server
          folders:
          - folder_name: database
            files:
            - file_name: database.db
          services:
          - service_name: DatabaseService
        - node_name: router_1

        max_folders_per_node: 2
        max_files_per_folder: 2
        max_services_per_node: 2
        max_nics_per_node: 8
        max_acl_rules: 10
        ip_list:
          - 192.168.10.21
          - 192.168.1.12
        wildcard_list:
          - 0.0.0.1
    reward_function:
      reward_components:
        - type: DUMMY

    agent_settings:
      flatten_obs: False
"""
custom_blue = yaml.safe_load(custom_blue_agent_yaml)

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = custom_blue
    

blue_env = PrimaiteGymEnv(env_config=cfg)

In [ ]:
# Utility function for showing OBS changes between each time step.

from deepdiff.diff import DeepDiff

def display_obs_diffs(old, new, step_counter):
    """
    Use DeepDiff to extract and display differences in old and new instances of
    the observation space.

    :param old: observation space instance.
    :param new: observation space instance.
    :param step_counter: current step counter.
    """
    print("\nObservation space differences")
    print("-----------------------------")
    diff = DeepDiff(old, new)
    print(f"Step {step_counter}")
    for d,v in diff.get('values_changed', {}).items():
        print(f"{d}: {v['old_value']} -> {v['new_value']}")

### **Command and Control** | Blue Agent Relevance | Observation Space

This section demonstrates the impacts that each of that the C2 Beacon and the C2 Server's commands cause on the observation space (OBS).

#### **Command and Control** | OBS Impact | C2 Beacon | Installation & Configuration

In [ ]:
# Resetting the environment and capturing the default observation space.
blue_env.reset()
default_obs, _, _, _, _ = blue_env.step(0)

In [ ]:
# Setting up the C2 Suite via the simulation API.

client_1: Computer = blue_env.game.simulation.network.get_node_by_hostname("client_1")
web_server: Server = blue_env.game.simulation.network.get_node_by_hostname("web_server")

# Installing the C2 Server.
client_1.software_manager.install(C2Server)
c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server.run()

# Installing the C2 Beacon.
web_server.software_manager.install(C2Beacon)
c2_beacon: C2Beacon = web_server.software_manager.software["C2Beacon"]
c2_beacon.configure(c2_server_ip_address="192.168.10.21")
c2_beacon.establish()

In [ ]:
# Capturing the observation impacts of the previous code cell: C2 Suite setup.
c2_configuration_obs, _, _, _, _ = blue_env.step(0)

In [ ]:
display_obs_diffs(default_obs, c2_configuration_obs, blue_env.game.step_counter)

#### **Command and Control** | OBS Impact | C2 Server | Terminal Command

Using the C2 Server's ``TERMINAL`` command it is possible to install a ``RansomwareScript`` application onto the C2 Beacon's host.

The below code cells perform this as well as capturing the OBS impacts.

It's important to note that the ``TERMINAL`` command is not limited to just installing software.

In [ ]:
# Installing RansomwareScript via C2 Terminal Commands
ransomware_install_command = {"commands":[["software_manager", "application", "install", "RansomwareScript"]],
                              "username": "admin",
                              "password": "admin"}
c2_server.send_command(C2Command.TERMINAL, command_options=ransomware_install_command)


In [ ]:
# Configuring the RansomwareScript
ransomware_config = {"server_ip_address": "192.168.1.14", "payload": "ENCRYPT"}
c2_server.send_command(C2Command.RANSOMWARE_CONFIGURE, command_options=ransomware_config)

In [ ]:
# Capturing the observation impacts of the previous code cell: Ransomware installation & configuration.
c2_ransomware_obs, _, _, _, _ = blue_env.step(0)

In [ ]:
display_obs_diffs(default_obs, c2_ransomware_obs, env.game.step_counter)

#### **Command and Control** | OBS Impact | C2 Server | Data Exfiltration

Before encrypting the database.db file, the ``DATA_EXFILTRATION`` command can be used to copy the database.db file onto both the C2 Server and the C2 Beacon's file systems:

In [ ]:
exfil_options={
    "username": "admin",
    "password": "admin",
    "target_ip_address": "192.168.1.14",
    "target_folder_name": "database",
    "exfiltration_folder_name": "exfiltration_folder",
    "target_file_name": "database.db",
}

In [ ]:
c2_server.send_command(given_command=C2Command.DATA_EXFILTRATION, command_options=exfil_options)

In [ ]:
c2_exfil_obs, _, _, _, _ = blue_env.step(0)

In [ ]:
display_obs_diffs(c2_ransomware_obs, c2_exfil_obs, env.game.step_counter)

#### **Command and Control** | OBS Impact | C2 Server | Ransomware Commands

The code cell below demonstrates the differences between the ransomware script installation obs and the impact of RansomwareScript upon the database.

In [ ]:
# Configuring the RansomwareScript
ransomware_config = {"server_ip_address": "192.168.1.14", "payload": "ENCRYPT"}
c2_server.send_command(C2Command.RANSOMWARE_CONFIGURE, command_options=ransomware_config)

In [ ]:
# Waiting for the ransomware to finish installing and then launching the RansomwareScript.
blue_env.step(0)
c2_server.send_command(C2Command.RANSOMWARE_LAUNCH, command_options={})

In [ ]:
# Capturing the observation impacts of the previous code cell: Launching the RansomwareScript.
c2_final_obs, _, _, _, _ = blue_env.step(0)

In [ ]:
display_obs_diffs(c2_ransomware_obs, c2_final_obs, blue_env.game.step_counter)

### **Command and Control** | Blue Agent Relevance | Action Space

The next section of this notebook will go over some potential blue agent actions that could be use to thwart the previously demonstrated attack.

In [ ]:
# This method is used to simplify setting up the C2Server and the C2 Beacon.
def c2_setup(given_env: PrimaiteGymEnv):
    client_1: Computer = given_env.game.simulation.network.get_node_by_hostname("client_1")
    web_server: Server = given_env.game.simulation.network.get_node_by_hostname("web_server")

    client_1.software_manager.install(C2Server)
    c2_server: C2Server = client_1.software_manager.software["C2Server"]
    c2_server.run()

    web_server.software_manager.install(C2Beacon)
    c2_beacon: C2Beacon = web_server.software_manager.software["C2Beacon"]
    c2_beacon.configure(c2_server_ip_address="192.168.10.21")
    c2_beacon.establish()

    return given_env, c2_server, c2_beacon, client_1, web_server

#### Removing the C2 Beacon.

The simplest way a blue agent could prevent the C2 suite is by simply removing the C2 beacon from it's installation point. 

In [ ]:
blue_env.reset()

In [ ]:
# Setting up the C2 Suite using the c2_setup method & capturing the OBS impacts

blue_env, c2_server, c2_beacon, client_1, web_server = c2_setup(given_env=blue_env)
pre_blue_action_obs, _, _, _, _ = blue_env.step(0)

The code cell below uses the custom blue agent defined at the start of this section perform a NODE_APPLICATION_REMOVE on the C2 beacon:

In [ ]:
# Using CAOS ACTION: NODE_APPLICATION_REMOVE & capturing the OBS
post_blue_action_obs, _, _, _, _ = blue_env.step(1)

Which we can see after the effects of after stepping another timestep and looking at the web_servers software manager and the OBS differences.

In [ ]:
blue_env.step(0)
web_server.software_manager.show()

In [ ]:
display_obs_diffs(pre_blue_action_obs, post_blue_action_obs, blue_env.game.step_counter)

Now we are unable to do so as the C2 Server has lost it's connection to the C2 Beacon:

In [ ]:
# Attempting to install the C2 RansomwareScript
ransomware_install_command = {"commands":[["software_manager", "application", "install", "RansomwareScript"]],
                            "username": "admin",
                            "password": "admin"}

c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server.send_command(C2Command.TERMINAL, command_options=ransomware_install_command)

#### Shutting down the node infected with a C2 Beacon.

Another way a blue agent can prevent the C2 suite is by shutting down the C2 beacon's host node. Whilst not as effective as the previous option, depending on the situation (such as multiple malicious applications) or other scenarios it may be more timestep efficient for a blue agent to shut down a node directly.

In [ ]:
blue_env.reset()

In [ ]:
# Setting up the C2 Suite using the c2_setup method & capturing the OBS impacts

blue_env, c2_server, c2_beacon, client_1, web_server = c2_setup(given_env=blue_env)
pre_blue_action_obs, _, _, _, _ = blue_env.step(0)

The code cell below uses the custom blue agent defined at the start of this section to perform a ``NODE_SHUT_DOWN`` action on the web server.

In [ ]:
# Using CAOS ACTION: NODE_SHUT_DOWN & capturing the OBS
post_blue_action_obs, _, _, _, _ = blue_env.step(2)

Which we can see the effects of after another timestep and looking at the web server's operating state & the OBS differences.

In [ ]:
web_server = blue_env.game.simulation.network.get_node_by_hostname("web_server")
print(web_server.operating_state)

In [ ]:
display_obs_diffs(pre_blue_action_obs, post_blue_action_obs, blue_env.game.step_counter)

In [ ]:
# Attempting to install the C2 RansomwareScript
ransomware_install_command = {"commands":["software_manager", "application", "install", "RansomwareScript"],
                            "username": "admin",
                            "password": "admin"}

c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server.send_command(C2Command.TERMINAL, command_options=ransomware_install_command)

#### Blocking C2 Traffic via ACL.

Another potential option a blue agent could take is by placing an ACL rule which blocks traffic between the C2 Server and C2 Beacon.

It's worth noting the potential effectiveness of this approach is connected to the current green agent traffic on the network. For example, if there are multiple green agents using the C2 Beacon's host node then blocking all traffic would lead to a negative reward. The same applies for the previous example.

In [ ]:
blue_env.reset()

In [ ]:
# Setting up the C2 Suite using the c2_setup method & capturing the OBS impacts

blue_env, c2_server, c2_beacon, client_1, web_server = c2_setup(given_env=blue_env)
pre_blue_action_obs, _, _, _, _ = blue_env.step(0)

The code cell below uses the custom blue agent defined at the start of this section to perform a ROUTER_ACL_ADDRULE on router 1.

In [ ]:
# Using CAOS ACTION: ROUTER_ACL_ADDRULE & capturing the OBS
post_blue_action_obs, _, _, _, _ = blue_env.step(3)

Which we can see the effects of after another timestep and looking at router 1's ACLs and the OBS differences.

In [ ]:
router_1: Router = blue_env.game.simulation.network.get_node_by_hostname("router_1")
router_1.acl.show()

Now we can see that the C2 applications are unable to maintain connection - thus being unable to execute correctly.

In [ ]:
blue_env.step(0)

# Attempting to install and execute the ransomware script
c2_server.send_command(C2Command.TERMINAL, command_options=ransomware_install_command)
c2_server.send_command(C2Command.RANSOMWARE_LAUNCH, command_options={})

In [ ]:
router_1.acl.show()

Because of the ACL rule the C2 beacon never received the ransomware installation and execute commands from the C2 server:

In [ ]:
web_server.software_manager.show()

In [ ]:
database_server: Server = blue_env.game.simulation.network.get_node_by_hostname("database_server")
database_server.software_manager.file_system.show(full=True)

In [ ]:
display_obs_diffs(pre_blue_action_obs, post_blue_action_obs, blue_env.game.step_counter)

## **Command and Control** | Configurability 

This section of the notebook demonstrates the C2 configuration options and their impact on the simulation layer and the game layer.

The table below is the currently offered C2 Beacon configuration options:

|Configuration Option | Option Meaning                                                            |Default Option | Type    | _Optional_ |
|---------------------|---------------------------------------------------------------------------|---------------|---------|------------|
|c2_server_ip_address | The IP Address of the C2 Server. (The C2 Server must be running)          |_None_         |str (IP) | _No_       |
|keep_alive_frequency | How often should the C2 Beacon confirm it's connection in timesteps.      |5              |Int      | _Yes_      |
|masquerade_port      | What port should the C2 traffic use? (TCP or UDP)                         |TCP            |Str      | _Yes_      |
|masquerade_protocol  | What protocol should the C2 traffic masquerade as? (HTTP, FTP or DNS)     |HTTP           |Str      | _Yes_      |

The C2 Server currently does not offer any unique configuration options. The C2 Server aligns itself with the C2 Beacon's configuration options once connection is established.

As demonstrated earlier, red agents can use the ``CONFIGURE_C2_BEACON`` action to configure these settings mid episode through the configuration options:

``` YAML
...
    action: CONFIGURE_C2_BEACON
        options:
        node_id: 0
        config:
            c2_server_ip_address: 192.168.10.21
            keep_alive_frequency: 10
            masquerade_protocol: TCP
            masquerade_port: DNS
```

### **Command and Control** | Configurability | C2 Server IP Address

As with a majority of client and server based application configuration in primaite, the remote IP of a server must be supplied.

In the case of the C2 Beacon, the C2 Server's IP address must be supplied before the C2 beacon will be able to perform any other actions (including ``APPLICATION EXECUTE``).

If the network contains multiple C2 Servers then it's also possible to switch to a different C2 server mid-episode which is demonstrated in the below code cells.

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = c2_agent_yaml
    

c2_config_env = PrimaiteGymEnv(env_config=cfg)

Installing the C2 Server on both client 1 and client 2.

In [ ]:
web_server: Server = c2_config_env.game.simulation.network.get_node_by_hostname("web_server")
web_server.software_manager.install(C2Beacon)
c2_beacon: C2Beacon = web_server.software_manager.software["C2Beacon"]

client_1: Computer = c2_config_env.game.simulation.network.get_node_by_hostname("client_1")
client_1.software_manager.install(C2Server)
c2_server_1: C2Server = client_1.software_manager.software["C2Server"]
c2_server_1.run()

client_2: Computer = c2_config_env.game.simulation.network.get_node_by_hostname("client_2")
client_2.software_manager.install(C2Server)
c2_server_2: C2Server = client_2.software_manager.software["C2Server"]
c2_server_2.run()

Configuring the C2 Beacon to establish connection to the C2 Server on client_1 (192.168.10.21)

In [ ]:
env.step(2) # Agent Action Equivalent to c2_beacon.configure(c2_server_ip_address="192.168.10.21")
env.step(3) # Agent action Equivalent to c2_beacon.establish()
c2_beacon.show()
c2_server_1.show()

Now reconfiguring the C2 Beacon to establish connection to the C2 Server on client_2 (192.168.10.22)

In [ ]:
env.step(9) # Equivalent of to c2_beacon.configure(c2_server_ip_address="192.168.10.22")
env.step(3)

c2_beacon.show()
c2_server_2.show()

After six timesteps the client_1 server will recognise the C2 beacon's previous connection as dead and clear its connections. (This is dependent on the ``Keep Alive Frequency`` setting.)

In [ ]:
for i in range(6):
    env.step(0)
    
c2_server_1.show()

### **Command and Control** | Configurability | Keep Alive Frequency

In order to confirm it's connection the  C2 Beacon will send out a ``Keep Alive`` to the C2 Server and receive a keep alive back. 

By default, this occurs every 5 timesteps. However, this setting can be configured to be much more infrequent or as frequent as every timestep. 

The next set of code cells below demonstrate the impact that this setting has on blue agent observation space.

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = custom_blue
    cfg['agents'][0]['observation_space']['options']['components'][0]['options']['num_ports'] = 3
    cfg['agents'][0]['observation_space']['options']['components'][0]['options']['monitored_traffic'].update({"tcp": ["HTTP","FTP"]})
    cfg['agents'][0]['observation_space']['options']['components'][0]['options']['monitored_traffic'].update({"udp": ["DNS"]})

blue_config_env = PrimaiteGymEnv(env_config=cfg)

In [ ]:
# Performing the usual c2 setup:
blue_config_env, c2_server, c2_beacon, client_1, web_server = c2_setup(given_env=blue_config_env)

# Flushing out the OBS impacts from setting up the C2 suite.
blue_config_env.step(0)
blue_config_env.step(0)

# Capturing the 'default' obs (Post C2 installation and configuration):
default_obs, _, _, _, _ = blue_config_env.step(0)

The next code cells capture the obs impact of the default Keep Alive Frequency which is 5 timesteps:

In [ ]:
c2_beacon.configure(c2_server_ip_address="192.168.10.21")
c2_beacon.establish()
c2_beacon.show()

The code cell below executes 10 timesteps and displays the differences between the default and the current timestep.

You will notice that the only two timesteps displayed observation space differences. This is due to the C2 Suite confirming their connection through sending ``Keep Alive`` traffic across the network every 5 timesteps.

In [ ]:
for i in range(10):
    keep_alive_obs, _, _, _, _ = blue_config_env.step(0)
    display_obs_diffs(default_obs, keep_alive_obs, blue_config_env.game.step_counter)

Next, the code cells below configure the C2 Beacon to confirm connection on every timestep via changing the ``keep_alive_frequency`` to ``1``.

In [ ]:
c2_beacon.configure(c2_server_ip_address="192.168.10.21", keep_alive_frequency=1)
c2_beacon.establish()
c2_beacon.show()

Demonstrating that the observation impacts of the Keep Alive can be seen on every timestep:

In [ ]:
# Comparing the OBS of the default frequency to a timestep frequency of 1 
for i in range(2):
    keep_alive_obs, _, _, _, _ = blue_config_env.step(0)
    display_obs_diffs(default_obs, keep_alive_obs, blue_config_env.game.step_counter)

Lastly, the keep_alive_frequency can also be used to configure the C2 Beacon to confirm connection less frequently. 

The code cells below demonstrate the impacts of changing the frequency rate to ``7`` timesteps.

In [ ]:
c2_beacon.configure(c2_server_ip_address="192.168.10.21", keep_alive_frequency=7)

# Comparing the OBS of the default frequency to a timestep frequency of 7
for i in range(7):
    keep_alive_obs, _, _, _, _ = blue_config_env.step(0)
    display_obs_diffs(default_obs, keep_alive_obs, blue_config_env.game.step_counter)

### **Command and Control** | Configurability | Masquerade Port & Masquerade Protocol

The final configurable options are ``Masquerade Port`` & ``Masquerade Protocol``. These options can be used to control the networking IP Protocol and Port the C2 traffic is currently using.

In the real world, adversaries take defensive steps to reduce the chance that an installed C2 Beacon is discovered. One of the most commonly used methods is to masquerade C2 traffic as other commonly used networking protocols.

In primAITE, red agents can begin to simulate stealth behaviour by configuring C2 traffic to use different protocols mid episode or between episodes.

Currently, red agent actions support the following port and protocol options:

| Supported Ports  | Supported Protocols |
|------------------|---------------------|
|``DNS``           | ``UDP``             |
|``FTP``           | ``TCP``             |
|``HTTP``          |                     |



The next set of code cells will demonstrate the impact of this option from a blue agent perspective.

In [ ]:
blue_config_env.reset()

# Performing the usual c2 setup:
blue_config_env, c2_server, c2_beacon, client_1, web_server = c2_setup(given_env=blue_config_env)

blue_config_env.step(0)

# Capturing the 'default' obs (Post C2 installation and configuration):
default_obs, _, _, _, _ = blue_config_env.step(0)

By default, the C2 suite will masquerade a Web Browser, meaning C2 Traffic will opt to use ``TCP`` and ``HTTP`` (Port 80):

In [ ]:
# Capturing default C2 Traffic 
for i in range(3):
    tcp_c2_obs, _, _, _, _ = blue_config_env.step(0)

display_obs_diffs(default_obs, tcp_c2_obs, blue_config_env.game.step_counter)

However, C2 Beacon can be configured to use UDP (``Masquerade Protocol``) and we can also configure the C2 Beacon to use a different Port (``Masquerade Port``) for example ``DNS``.  

In [ ]:
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.network.transmission.transport_layer import Port
# As we're configuring via the PrimAITE API we need to pass the actual IPProtocol/Port (Agents leverage the simulation via the game layer and thus can pass strings).
c2_beacon.configure(c2_server_ip_address="192.168.10.21", masquerade_protocol=IPProtocol.UDP,  masquerade_port=Port.DNS)
c2_beacon.establish()
c2_beacon.show()

In [ ]:
# Capturing UDP C2 Traffic
for i in range(5):
    udp_c2_obs, _, _, _, _ = blue_config_env.step(0)

display_obs_diffs(tcp_c2_obs, udp_c2_obs, blue_config_env.game.step_counter)